#Import

In [ ]:
import os
import pandas as pd
import math
from numpy.ma.core import mean
from pylab import text
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from pylab import text
%pip install xlsxwriter
import xlsxwriter
from google.colab import files
import statistics
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn

#Save Results

In [ ]:
def save_excel_dumbbells(df1, df2, df3, df4, df5, df6):
    title = 'Size Distributions.xlsx'
    writer = pd.ExcelWriter(title, engine='xlsxwriter')
    df1.to_excel(writer, sheet_name='Abstract')
    df2.to_excel(writer, sheet_name='Spheres')
    df3.to_excel(writer, sheet_name='Dumbbells')
    df4.to_excel(writer, sheet_name='Lobes')
    df5.to_excel(writer, sheet_name='Large Lobes')
    df6.to_excel(writer, sheet_name='Small Lobes')
    writer.save()
    files.download(title)
    return

def save_excel_tripods(df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13):
    title = 'Size Distributions.xlsx'
    writer = pd.ExcelWriter(title, engine='xlsxwriter')
    df1.to_excel(writer, sheet_name='Abstract')
    df2.to_excel(writer, sheet_name='Tripods')
    df3.to_excel(writer, sheet_name='Linear Tripods')
    df4.to_excel(writer, sheet_name='Triangular Tripods')
    df5.to_excel(writer, sheet_name='Large Tripod Extremities')
    df6.to_excel(writer, sheet_name='Middle Lobe')
    df7.to_excel(writer, sheet_name='Small Tripod Extremities')
    df8.to_excel(writer, sheet_name='Largest Lobes - No Positioning')
    df9.to_excel(writer, sheet_name='Medium Lobes - No Positioning')
    df10.to_excel(writer, sheet_name='Smallest Lobes - No Positioning')
    df11.to_excel(writer, sheet_name='Remaining Dumbbells')
    df12.to_excel(writer, sheet_name='Dumbbells - Large Lobes')
    df13.to_excel(writer, sheet_name='Dumbbells - Small Lobes')
    writer.save()
    files.download(title)
    return

#Treat Data - Dumbbells and Spheres

In [ ]:
#returns a list with the name of all uploaded csv files
table_list, csv_list = [], []
for filename in os.listdir('/content'):
    if filename.endswith('.csv'):
        table_list.append(pd.read_csv(filename,sep="|"))
        csv_list.append(filename.split(".")[0])

all_list, watershed_list = [], []

for i in range(len(csv_list)): #enters two lists at once, the list with the particles and the list with the divided particles
    if 'Watershed' in csv_list[i]:
        watershed_list.append(csv_list[i])
    else:
        all_list.append(csv_list[i])

all_list.sort()
watershed_list.sort()

image_list = [] #creates a list of the number of the images to make them identifiable using the function rearrange_columns
for i  in range(len(all_list)):
    if '_' in all_list[i]:
        image_list.append(all_list[i][-10 : -8])
    else:
        image_list.append(all_list[i][-10 : -8])

all_df, watershed_df = [], []

for i in range (len(all_list)): #transforms the uploaded lists into dataframes
    all_df.append('all_df' + str(i))
    watershed_df.append('watershed_df' + str(i))
    all_df[i] = pd.read_csv(all_list[i] + '.csv', decimal = '.')
    watershed_df[i] = pd.read_csv(watershed_list[i] + '.csv', decimal = '.')

for j in range(len(all_df)): #deletes the measurements that show particles that are too large, aka overlapping particles
    droplist = []
    if len(watershed_df[j]) > len(all_df[j]):
        for i in range(0, len(all_df[j])):
            NaNlist = []
            if all_df[j].loc[i]['Area'] > 1.5*all_df[j]['Area'].mean() or all_df[j].loc[i]['Area'] < 0.5*all_df[j]['Area'].mean():
                for k in range(len(all_df[j].columns)):
                    NaNlist.append(np.NaN)
                all_df[j].loc[i] = NaNlist

                #watershed_df[j] = watershed_df[j].drop([i], axis=1, inplace=True)
                #all_df[j] = all_df[j].drop([i], axis=1, inplace=True)



#with this section all the spheres and all the dumbbells are identified and classified and a distribution can be plotted
sphere_list, dumbbell_list = [], []
sphere_image, dumbbell_image = [], []##########################################################################################################################
for j in range(len(all_df)):
    for i in range(len(all_df[j])):
        #if all_df[j].iloc[i]['Round'] > 0.85 and all_df[j].iloc[i]['AR'] < 1.15:
        #if all_df[j].iloc[i]['Round'] > 0.85 and all_df[j].iloc[i]['AR'] < 1.15 and all_df[j].iloc[i]['Circ.'] > 0.85:
        if all_df[j].iloc[i]['Round'] > 0.85 and all_df[j].iloc[i]['AR'] < 1.15 and all_df[j].iloc[i]['Circ.'] > 0.85:
            sphere_list.append((all_df[j].iloc[i])) #list of spheres
            sphere_image.append(j) #list of identifiers, used in function rearrenge_columns
        elif all_df[j].iloc[i]['AR'] > 1.4:
            dumbbell_list.append((all_df[j].iloc[i])) #list of dumbbells
            dumbbell_image.append(j) #list of identifiers, used in function rearrenge_columns

lobe_list, lobe_image = [], []



aux_list = [] #creates a dictionary to identify the images on which each particle was observed
for i in range(len(all_df)):
    aux_list.append(i)
image_dictionary = dict(zip(aux_list, image_list))


def rearrange_columns (particle_list, imagenr, dictionary): #adds identifies to the dataframes containing the data about the particles
#and also rearranges the columns
#and calculates the diameter of the particles
    df = pd.DataFrame(particle_list)
    particle_image = []
    df.rename(columns = {' ':'Particle'}, inplace = True)
    df['Particle']
    for i in range(len(df)):
        particle_image.append(dictionary[imagenr[i]])
    df['Image'] = particle_image
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    diameter = []
    volume = []
    for i in range(len(df)):
        #diameter.append(2*math.sqrt(df['Area'].iloc[i]/math.pi))
        diameter.append(df['Major'].iloc[i])
        volume.append(4*math.pi*(((diameter[-1])/2)**3)/3)
    df['Diameter'] = diameter
    df['Volume'] = volume
    return df

if len(sphere_list) > 0:
    sphere_df = rearrange_columns (sphere_list, sphere_image, image_dictionary)
    sphere_df.reset_index(drop=True, inplace=True)

if len(dumbbell_list) > 0:
    dumbbell_df = rearrange_columns (dumbbell_list, dumbbell_image, image_dictionary)
    dumbbell_df.reset_index(drop=True, inplace=True)

lobe_list, lobe_image = [], []
for j in range(len(watershed_df)):
    for i in range(len(watershed_df[j])):
        lobe_list.append((watershed_df[j].iloc[i])) #list of dumbbell lobes, still includes the sphres
        lobe_image.append(j)

lobe_df = rearrange_columns (lobe_list, lobe_image, image_dictionary)
lobe_df.reset_index(drop=True, inplace=True)
print ('Total input lobes = ' + str(len(lobe_df)))

copialista = [] #function that uses the created list of dumbbell lobes and the list of identified spheres to eliminate the spheres
#with that only the dumbbell lobes are present at the end, but still not assigned to their respective particles
for i in range(len(lobe_df)):
    x = True
    for j in range(len(sphere_list)):
        if lobe_df['Image'].iloc[i] == sphere_df['Image'].iloc[j] and sphere_df['X'].iloc[j] + (sphere_df['Minor'].iloc[j])/2 < lobe_df['X'].iloc[i] < sphere_df['X'].iloc[j] - (sphere_df['Minor'].iloc[j])/2 and sphere_df['Y'].iloc[j] - (sphere_df['Minor'].iloc[j])/2 < lobe_df['Y'].iloc[i] < sphere_df['Y'].iloc[j] + (sphere_df['Minor'].iloc[j])/2:
            x = False
    copialista.append(x)

lobe_df = lobe_df[copialista] #drops all the spheres
lobe_df.reset_index(drop=True, inplace=True)
print ('Total identified lobes = ' + str(len(lobe_df)))

lobe_positioning = []
lobe_relationing = []
copialista = []

if len(dumbbell_list) > 0:
    for i in range(len(lobe_df)): #function uses the distance between the dumbbell lobes and the whole dumbbells to determine to which dumbbells the lobes belong
        x = False
        for j in range(len(dumbbell_df)):
            if x == False:
                if lobe_df['Image'].iloc[i] == dumbbell_df['Image'].iloc[j] and dumbbell_df['X'].iloc[j] - dumbbell_df['Minor'].iloc[j]/2 < lobe_df['X'].iloc[i] < dumbbell_df['X'].iloc[j] + dumbbell_df['Minor'].iloc[j]/2 and dumbbell_df['Y'].iloc[j] - dumbbell_df['Minor'].iloc[j]/2 < lobe_df['Y'].iloc[i] < dumbbell_df['Y'].iloc[j] + dumbbell_df['Minor'].iloc[j]/2:
                    lobe_positioning.append([dumbbell_df['Image'].iloc[j], dumbbell_df['Particle'].iloc[j]])
                    lobe_relationing.append(dumbbell_df['Particle'].iloc[j])
                    x = True
                else:
                    x = False
        copialista.append(x)
    lobe_df = lobe_df[copialista] #drops all the spheres
    lobe_df.reset_index(drop=True, inplace=True)

    lobe_df['Dumbbell'] = lobe_relationing
    lobe_df['Large'] = False
    lobe_df['Small'] = False

    for i in range(len(lobe_positioning)): #identifies to which other half the lobe belongs
        for j in range(i+1, len(lobe_positioning)):
            if lobe_positioning[i] == lobe_positioning[j]:
                if lobe_df['Area'].iloc[i] > lobe_df['Area'].iloc[j]:
                    lobe_df['Large'].iloc[i] = True
                    lobe_df['Small'].iloc[j] = True
                if lobe_df['Area'].iloc[i] < lobe_df['Area'].iloc[j]:
                    lobe_df['Small'].iloc[i] = True
                    lobe_df['Large'].iloc[j] = True
    droplist = []
    for i in range(len(lobe_df)):
        if lobe_df['Large'].iloc[i] == False and lobe_df['Small'].iloc[i] == False:
            droplist.append(False)
        else: droplist.append(True)

    lobe_df = lobe_df[droplist]

    largelobe_df = lobe_df[lobe_df['Large']]
    smalllobe_df = lobe_df[lobe_df['Small']]

    cols = list(largelobe_df)
    cols = cols[0:2] + [cols[-3]] + cols[2:-3] + cols[-2:]
    lobe_df = lobe_df[cols]
    largelobe_df = largelobe_df[cols]
    smalllobe_df = smalllobe_df[cols]

def find_distances_dumbbells(dumbbell_df, largelobe_df, smalllobe_df):
    def dist (a, b): #calcula a distancia entre dois pontos por geometria euclidiana
        dist = math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
        return dist
    points_list = []
    for i in range(len(largelobe_df)):
        for j in range(len(smalllobe_df)): #acha qual o outro lado do dumbbell partindo do maior lado
            if largelobe_df['Image'].iloc[i] == smalllobe_df['Image'].iloc[j] and largelobe_df['Dumbbell'].iloc[i] == smalllobe_df['Dumbbell'].iloc[j]:
                points_list.append([largelobe_df['X'].iloc[i], largelobe_df['Y'].iloc[i], smalllobe_df['X'].iloc[j], smalllobe_df['Y'].iloc[j], largelobe_df['Image'].iloc[i], largelobe_df['Dumbbell'].iloc[i]])
    #aqui também são adicionados o numero da partícula e em qual imagem ela foi identificada

    distance_list = []
    for i in range(len(points_list)):
        distance_list.append(dist((points_list[i][0], points_list[i][1]),(points_list[i][2], points_list[i][3]))) #aqui as distancias sao medidas e a identificacao da particula entra na lista apos a ditancia

    dumbbell_df['Distance'] = ''
    for i in range(len(dumbbell_df)):
        for j in range(len(points_list)):
            if dumbbell_df['Image'].iloc[i] == points_list[j][4] and dumbbell_df['Particle'].iloc[i] == points_list[j][5]:
                dumbbell_df['Distance'][i] = distance_list[j]
        if dumbbell_df['Distance'].iloc[i] == '':
            dumbbell_df['Distance'].iloc[i] = np.NaN

    return dumbbell_df

def find_overlap_dumbbells(dumbbell_df, largelobe_df, smalllobe_df):
    dumbbell_df['Overlap'] = dumbbell_df['Major']
    dumbbell_df['Relative Overlap'] = dumbbell_df['Major']
    for i in range(len(dumbbell_df)):
        if np.isnan(dumbbell_df['Distance'].iloc[i]):
            dumbbell_df['Overlap'].iloc[i] = np.NaN
            dumbbell_df['Relative Overlap'].iloc[i] = np.NaN

        else:
            for j in range(len(largelobe_df)):
                if dumbbell_df['Particle'].iloc[i] == largelobe_df['Dumbbell'].iloc[j]:
                    if dumbbell_df['Image'].iloc[i] == largelobe_df['Image'].iloc[j]:
                        dumbbell_df['Overlap'].iloc[i] = largelobe_df['Diameter'].iloc[j]
            for j in range(len(smalllobe_df)):
                if dumbbell_df['Particle'].iloc[i] == smalllobe_df['Dumbbell'].iloc[j]:
                    if dumbbell_df['Image'].iloc[i] == smalllobe_df['Image'].iloc[j]:
                        dumbbell_df['Overlap'].iloc[i] = dumbbell_df['Overlap'].iloc[i] + smalllobe_df['Diameter'].iloc[j]

            dumbbell_df['Overlap'].iloc[i] = dumbbell_df['Overlap'].iloc[i] - dumbbell_df['Major'].iloc[i]
            dumbbell_df['Relative Overlap'].iloc[i] = dumbbell_df['Overlap'].iloc[i] / dumbbell_df['Major'].iloc[i]

    return dumbbell_df


def corrected_distances_dumbbells(dumbbell_df, largelobe_df, smalllobe_df):
    from tqdm import tqdm
    positioning = []
    for i in tqdm(range(len(dumbbell_df))):
        for j in range(len(largelobe_df)):
            if dumbbell_df['Particle'].iloc[i] == largelobe_df['Dumbbell'].iloc[j]:
                if dumbbell_df['Image'].iloc[i] == largelobe_df['Image'].iloc[j]:
                    positioning.append([i,j])
        for k in range(len(smalllobe_df)):
            if dumbbell_df['Particle'].iloc[i] == smalllobe_df['Dumbbell'].iloc[k]:
                if dumbbell_df['Image'].iloc[i] == smalllobe_df['Image'].iloc[k]:
                    positioning[-1].append(k)
                    positioning[-1].append(dumbbell_df['Image'].iloc[i])
                    positioning[-1].append(dumbbell_df['Particle'].iloc[i])
    #positioning[dumbbellindex,
    #           largeindex,
    #           smallindex,
    #           dumbbell_df['Particle'].iloc[dumbbellindex],
    #           dumbbell_df['Image'].iloc[dumbbellindex]]

    #dumbbell_df['Corrected Distance'] = np.NaN
    #for i in tqdm(range(len(positioning))):
    #    dumbbell_df['Corrected Distance'].iloc[positioning[i][0]] = dumbbell_df['Major'].iloc[positioning[i][0]] - largelobe_df['Diameter'].iloc[positioning[i][1]]/2 - smalllobe_df['Diameter'].iloc[positioning[i][2]]/2
    #corrected_distance_list = list(dumbbell_df['Corrected Distance'])
    return dumbbell_df

if len(dumbbell_list) > 0:
    dumbbell_df = find_distances_dumbbells(dumbbell_df, largelobe_df, smalllobe_df)
    dumbbell_df = find_overlap_dumbbells(dumbbell_df, largelobe_df, smalllobe_df)
    #dumbbell_df = corrected_distances_dumbbells(dumbbell_df, largelobe_df, smalllobe_df)
    #dumbbell_df = dumbbell_df[0]

#creates a list with the most important parameters identified with the analysis

abstract_list = [['Number of spheres = ', '{:3f}'.format(len(sphere_list)), ''],
                 ['Fraction of spheres = ', round(len(sphere_list)*100/(len(sphere_list)+len(dumbbell_list)),2), '%'],
                ['Number of dumbbells = ', math.ceil(len(dumbbell_list)), ''],
                ['Fraction of dumbbells = ', round((len(dumbbell_list)*100/(len(sphere_list)+len(dumbbell_list))),2), '%']]

if len(sphere_list) > 2:
    mu1, std1 = norm.fit(sphere_df['Volume'])
    abstract_list.append(['Mean Sphere Diameter = ', round(mean(sphere_df['Diameter']), 3), 'nm'])
    abstract_list.append(['Sphere Standard Deviation = ', round(statistics.stdev(sphere_df['Diameter']), 3), 'nm'])
    abstract_list.append(['Mean Sphere Diameter, Weight Avg = ', round(2*((3*mean(sphere_df['Volume']))/(4*math.pi))**(1/3), 3), 'nm'])
    abstract_list.append(['Sphere Standard Deviation, Weight Avg = ', round(2*((3*(2*((3*mean(sphere_df['Volume']))/(4*math.pi))**(1/3)))/(4*math.pi))**(1/3), 3), 'nm'])
    abstract_list.append(['PDI = ', round((2*((3*mean(sphere_df['Volume']))/(4*math.pi))**(1/3))/(mean(sphere_df['Diameter'])), 7), 'nm'])

if dumbbell_list:
    if len(lobe_df) > 2:
        df = dumbbell_df.dropna()
        distancelist = np.array(df['Distance'], dtype=float)
        overlaplist = np.array(df['Overlap'], dtype=float)
        rel_overlaplist = np.array(df['Relative Overlap'], dtype=float)
        #abstract_list.append(['CORRECTED - Mean Dumbbell Lobe Distance = ', round(mean(list(dumbbell_df['Corrected Distance'])), 3), 'nm'])
        #abstract_list.append(['CORRECTED - Dumbbell Lobe Distance Standard Deviation = ', round(statistics.stdev(dumbbell_df['Corrected Distance']), 3), 'nm'])
        if len(distancelist) > 2:
            abstract_list.append(['Mean Dumbbell Lobe Distance = ', round(mean(distancelist), 3), 'nm'])
            abstract_list.append(['Dumbbell Lobe Distance Standard Deviation = ', round(statistics.stdev(distancelist), 3), 'nm'])

        if len(overlaplist) > 2:
            abstract_list.append(['Mean Dumbbell Lobe Overlap = ', round(mean(overlaplist), 3), 'nm'])
            abstract_list.append(['Dumbbell Lobe Overlap Standard Deviation = ', round(statistics.stdev(overlaplist), 3), 'nm'])

        if len(rel_overlaplist) > 2:
            abstract_list.append(['Mean Dumbbell Lobe Relative Overlap = ', round(mean(rel_overlaplist), 3), ''])
            abstract_list.append(['Dumbbell Lobe Relative Overlap Standard Deviation = ', round(statistics.stdev(rel_overlaplist), 3), ''])

        abstract_list.append(['Mean Dumbbell Lenght = ', round(mean(list(dumbbell_df['Major'])), 3)])
        abstract_list.append(['Dumbbell Lenght Standard Deviation = ', round(statistics.stdev(list(dumbbell_df['Major'])), 3), 'nm'])

        abstract_list.append(['Mean Dumbbell Lobe Diameter = ', round(mean(lobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Dumbbell Lobe Standard Deviation = ', round(statistics.stdev(lobe_df['Diameter']), 3), 'nm'])

        abstract_list.append(['Mean Lobe Diameter, Weight Avg = ', round(2*((3*mean(lobe_df['Volume']))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Lobe Diameter Standard Deviation, Weight Avg = ', round(2*((3*(2*((3*mean(lobe_df['Volume']))/(4*math.pi))**(1/3)))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Lobe PDI = ', round((2*((3*mean(lobe_df['Volume']))/(4*math.pi))**(1/3))/(mean(lobe_df['Diameter'])), 7), 'nm'])

        abstract_list.append(['Mean Large Dumbbell Lobe Diameter = ', round(mean(largelobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Large Dumbbell Lobe Standard Deviation = ', round(statistics.stdev(largelobe_df['Diameter']), 3), 'nm'])

        abstract_list.append(['Mean Large Lobe Diameter, Weight Avg = ', round(2*((3*mean(largelobe_df['Volume']))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Large Lobe Diameter Standard Deviation, Weight Avg = ', round(2*((3*(2*((3*mean(largelobe_df['Volume']))/(4*math.pi))**(1/3)))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Large Lobe PDI = ', round((2*((3*mean(largelobe_df['Volume']))/(4*math.pi))**(1/3))/(mean(largelobe_df['Diameter'])), 7), 'nm'])

        abstract_list.append(['Mean Small Dumbbell Lobe Diameter = ', round(mean(smalllobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Small Dumbbell Lobe Standard Deviation = ', round(statistics.stdev(smalllobe_df['Diameter']), 3), 'nm'])

        abstract_list.append(['Mean Small Lobe Diameter, Weight Avg = ', round(2*((3*mean(smalllobe_df['Volume']))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Small Lobe Diameter Standard Deviation, Weight Avg = ', round(2*((3*(2*((3*mean(smalllobe_df['Volume']))/(4*math.pi))**(1/3)))/(4*math.pi))**(1/3), 3), 'nm'])
        abstract_list.append(['Small Lobe PDI = ', round((2*((3*mean(smalllobe_df['Volume']))/(4*math.pi))**(1/3))/(mean(smalllobe_df['Diameter'])), 7), 'nm'])

        abstract_list.append(['Mean Dumbbell Aspect Ratio = ', round(mean(dumbbell_df['AR']), 3)])
        abstract_list.append(['Dumbbell Aspect Ratio Standard Deviation = ', round(statistics.stdev(dumbbell_df['AR']), 3)])
        abstract_list.append(['Mean Dumbbell Apparent Diameter = ', round(mean(dumbbell_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Dumbbell Apparent Diameter Standard Deviation = ', round(statistics.stdev(dumbbell_df['Diameter']), 3), 'nm'])

    elif 0 <= len(lobe_df) <= 2:
        df = dumbbell_df.dropna()
        distancelist = np.array(df['Distance'], dtype=float)
        abstract_list.append(['Mean Dumbbell Lobe Distance = ', round(mean(distancelist), 3)])
        abstract_list.append(['Dumbbell Lobe Distance Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Dumbbell Lenght = ', round(mean(list(dumbbell_df['Major'])), 3)])
        abstract_list.append(['Dumbbell Lenght Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Dumbbell Lobe Diameter = ', round(mean(lobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Dumbbell Lobe Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Large Dumbbell Lobe Diameter = ', round(mean(largelobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Large Dumbbell Lobe Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Small Dumbbell Lobe Diameter = ', round(mean(smalllobe_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Small Dumbbell Lobe Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Dumbbell Aspect Ratio = ', round(mean(dumbbell_df['AR']), 3)])
        abstract_list.append(['Dumbbell Aspect Ratio Standard Deviation = ', np.nan, 'nm'])
        abstract_list.append(['Mean Dumbbell Apparent Diameter = ', round(mean(dumbbell_df['Diameter']), 3), 'nm'])
        abstract_list.append(['Dumbbell Apparent Diameter Standard Deviation = ', np.nan, 'nm'])



abstract_df = pd.DataFrame(abstract_list)


if len(dumbbell_list) > 0 and len(sphere_list) > 0:
    save_excel_dumbbells(abstract_df, sphere_df, dumbbell_df, lobe_df, largelobe_df, smalllobe_df)
if len(dumbbell_list) > 0 and len(sphere_list) == 0:
    sphere_df = pd.DataFrame(columns = dumbbell_df.columns)
    save_excel_dumbbells(abstract_df, sphere_df, dumbbell_df, lobe_df, largelobe_df, smalllobe_df)
if len(sphere_list) > 0 and len(dumbbell_list) == 0:
    dumbbell_df = pd.DataFrame(columns = sphere_df.columns)
    save_excel_dumbbells(abstract_df, sphere_df, dumbbell_df, lobe_df, pd.DataFrame(), pd.DataFrame())

abstract_df

#Treat Data - Trimers

In [ ]:
#returns a list with the name of all uploaded csv files
table_list, csv_list = [], []
for filename in os.listdir('/content'):
    if filename.endswith('.csv'):
        table_list.append(pd.read_csv(filename,sep="|"))
        csv_list.append(filename.split(".")[0])

all_list, watershed_list = [], []

for i in range(len(csv_list)): #enters two lists at once, the list with the particles and the list with the divided particles
    if 'Watershed' in csv_list[i]:
        watershed_list.append(csv_list[i])
    else:
        all_list.append(csv_list[i])

all_list.sort()
watershed_list.sort()

image_list = [] #creates a list of the number of the images to make them identifiable using the function rearrange_columns
for i  in range(len(all_list)):
    if '_' in all_list[i]:
        image_list.append(all_list[i][-10 : -8])
    else:
        image_list.append(all_list[i][-10 : -8])

all_df, watershed_df = [], []

for i in range (len(all_list)): #transforms the uploaded lists into dataframes
    all_df.append('all_df' + str(i))
    watershed_df.append('watershed_df' + str(i))
    all_df[i] = pd.read_csv(all_list[i] + '.csv', decimal = '.')
    watershed_df[i] = pd.read_csv(watershed_list[i] + '.csv', decimal = '.', on_bad_lines='skip')

for j in range(len(all_df)): #deletes the measurements that show particles that are too large, aka overlapping particles
    for i in range(len(all_df[j])):
        if all_df[j].loc[i]['Area'] > 2*all_df[j]['Area'].mean():
            watershed_df[j] = watershed_df[j].drop([i])
            all_df[j] = all_df[j].drop([i])

def rearrange_columns (particle_list, imagenr, dictionary): #adds identifies to the dataframes containing the data about the particles
#and also rearranges the columns
#and calculates the diameter of the particles
    df = pd.DataFrame(particle_list)
    particle_image = []
    df.rename(columns = { ' ':'Particle'}, inplace = True)
    df['Particle']
    for i in range(len(df)):
        particle_image.append(dictionary[imagenr[i]])
    df['Image'] = particle_image
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    diameter = []
    volume = []
    for i in range(len(df)):
        diameter.append(df['Major'].iloc[i])
        volume.append(4*math.pi*(((diameter[-1])/2)**3)/3)
    df['Diameter'] = diameter
    df['Volume'] = volume
    return df

aux_list = [] #creates a dictionary to identify the images on which each particle was observed
for i in range(len(all_df)):
    aux_list.append(i)
image_dictionary = dict(zip(aux_list, image_list))

for i in range(len(all_df)): #sets the correct names for the images in which the particles were observed using imageJ
    all_df[i]['Image'] = image_dictionary[i]

lobe_list, lobe_image = [], []
for j in range(len(watershed_df)):
    for i in range(len(watershed_df[j])):
        lobe_list.append((watershed_df[j].iloc[i])) #list of tripod lobes, including triangular and linear particles
        lobe_image.append(j)
lobe_df = rearrange_columns (lobe_list, lobe_image, image_dictionary)
lobe_df.reset_index(drop=True, inplace=True)

for i in range(1, len(all_df)): #joins all of the measuments in only on dataframe
    all_df[0] = pd.concat([all_df[0], all_df[i]])

all_df = pd.DataFrame(all_df[0]) #ignores the measurements in other dataframes in the list, keeping only the one to which all were concatenated
cols = all_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
all_df = all_df[cols] #rearranges columns
all_df.reset_index(drop=True, inplace=True)
all_df.rename(columns = { ' ':'Particle'}, inplace = True)

#function checks if the lobes are contained inside the area covered by the whole particles
#so far the remaining dumbbells have not been separated from the tripods for analysis
lobe_positioning = []
lobe_fullpositioning = []
lobe_relationing = []
copialista = []
for i in range(len(lobe_df)): #identifies to which particle each lobe belongs depending on their positioning
    x = False
    for j in range(len(all_df)):
        if x == False:
            if lobe_df['Image'].iloc[i] == all_df['Image'].iloc[j] and all_df['BX'][j] < lobe_df['X'].iloc[i] < all_df['BX'][j]+all_df['Width'][j]*0.85 and all_df['BY'][j] < lobe_df['Y'].iloc[i] < all_df['BY'][j]+all_df['Height'][j]*0.85:
                lobe_positioning.append([all_df['Image'].iloc[j], all_df['Particle'].iloc[j]])
                lobe_fullpositioning.append([all_df['Image'].iloc[j], all_df['Particle'].iloc[j], lobe_df['Image'].iloc[i], lobe_df['Particle'].iloc[i]])
                lobe_relationing.append(all_df['Particle'].iloc[j])
                x = True
            else:
                x = False
    copialista.append(x)
print ('Number of input lobes: ' + str(len(lobe_df)))
lobe_df = lobe_df[copialista] #drops all the unidentified lobes
lobe_df.reset_index(drop=True, inplace=True)
lobe_df['Particle'] = lobe_relationing
print ('Number of identified lobes: ' + str(len(lobe_df)))

def dumbbell_or_tripod(all_df, lobe_df):
    sort_list = []
    for i in range(len(all_df)):
        sort_list.append([all_df['Image'].iloc[i], all_df['Particle'].iloc[i], 0]) #declares a list of lists containing the identifiers of each particle
                                                                                    #also has the third argument to be used to store the number of lobes found to allow for classification

    dumbbell_list, tripod_list = [], [] #classifies the particles as either dumbbell or tripod depending on how many lobes were found in it
    for i in range(len(sort_list)):
        for j in range(len(lobe_df)):
            if lobe_df['Image'].iloc[j] == sort_list[i][0] and lobe_df['Particle'].iloc[j] == sort_list[i][1]:
                sort_list[i][2] = sort_list[i][2]+1
        if sort_list[i][2] == 2 and all_df['AR'].iloc[i] < 1.8:
            sort_list[i][2] = 'Dumbbell'
            dumbbell_list.append(all_df.iloc[i])
        if sort_list[i][2] == 3:
            sort_list[i][2] = 'Tripod'
            tripod_list.append(all_df.iloc[i])

    dumbbell_df = pd.DataFrame(dumbbell_list)
    tripod_df = pd.DataFrame(tripod_list)

    lobe_df['Particle Sort'] = '' #works as a dictionary naming the sort of particle found
    for i in range(len(sort_list)):
        for j in range(len(lobe_df)):
            if sort_list[i][0] == lobe_df['Image'][j] and sort_list[i][1] == lobe_df['Particle'][j]:
                lobe_df['Particle Sort'][j] = sort_list[i][2]

    print ('Number of dumbbells = ' + str(math.ceil(len(dumbbell_list))) + '; ' + str(round((len(dumbbell_list)*100/(len(tripod_list)+len(dumbbell_list))),2)) + '%')
    print ('Number of tripods = ' + str(len(tripod_list)) + '; ' + str(round(len(tripod_list)*100/(len(tripod_list)+len(dumbbell_list)),2)) + '%')

    return lobe_df, dumbbell_df, tripod_df

lobe_df, dumbbell_df, tripod_df = dumbbell_or_tripod(all_df, lobe_df)

def measure(tripod_df, lobe_df):
    tripod_df['Classification'] = ''
    tripod_df['Angle'] = ''
    tripod_df['Large to Middle Distance'] = ''
    tripod_df['Small to Middle Distance'] = ''
    tripod_df['Large to Small Distance'] = ''

    triangular_list, linear_list = [], []
    for i in range(len(tripod_df)): #classifies the tripods as either triangular or linear depending on their aspect ratio
        if tripod_df['AR'].iloc[i] < 1.5:
            triangular_list.append(tripod_df.iloc[i])
            tripod_df['Classification'].iloc[i] = 'Triangular'
        if tripod_df['AR'].iloc[i] > 1.5:
            linear_list.append(tripod_df.iloc[i])
            tripod_df['Classification'].iloc[i] = 'Linear'

    lobe_tripod_list = [] #gathers the sizes of the three lobes present in each of the tripods
    for i in range(len(tripod_df)):
        aux = []
        for j in range(len(lobe_df)):
            if tripod_df['Image'].iloc[i] == lobe_df['Image'].iloc[j] and tripod_df['Particle'].iloc[i] == lobe_df['Particle'].iloc[j]:
                aux.append(j)
        lobe_tripod_list.append(aux)

    points_list = [] #gathers the coordintes for all the three lobes that compose the same tripod
    for i in range(len(tripod_df)):
        aux = []
        for j in range(len(lobe_df)):
            if tripod_df['Image'].iloc[i] == lobe_df['Image'].iloc[j] and tripod_df['Particle'].iloc[i] == lobe_df['Particle'].iloc[j]:
                aux.append([lobe_df['X'].iloc[j], lobe_df['Y'].iloc[j]])
        points_list.append(aux)

    tripod_df['Tripod Angle'] = '' #measures the angles between the three lobes comprising a given tripod, regardless of morphology
    lobe_df['Large Extremity'], lobe_df['Middle Lobe'], lobe_df['Small Extremity'] = False, False, False
    def getAngle(a, b, c):
        ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
        if ang < 0:
            ang = -ang
        if 180 < ang < 270:
            ang = ang-2*(ang-180)
        if ang > 270:
            ang = 360-ang
        return ang

    larger_to_middle_list = []
    smaller_to_middle_list = []
    larger_to_smaller_list = []

    def find_distances (points_list, i, larger, middle, smaller, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list):
        def dist (a, b):
            dist = math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
            return dist
        a = (lobe_df['X'].iloc[lobe_tripod_list[i][larger]], lobe_df['Y'].iloc[lobe_tripod_list[i][larger]])
        b = (lobe_df['X'].iloc[lobe_tripod_list[i][middle]], lobe_df['Y'].iloc[lobe_tripod_list[i][middle]])
        c = (lobe_df['X'].iloc[lobe_tripod_list[i][smaller]], lobe_df['Y'].iloc[lobe_tripod_list[i][smaller]])
        #dist_larger_to_middle = math.dist(a, b)
        #dist_smaller_to_middle = math.dist(b, c)
        #dist_larger_to_smaller = math.dist(a, c)
        larger_to_middle_list.append(dist(a, b))
        smaller_to_middle_list.append(dist(b, c))
        larger_to_smaller_list.append(dist(a, c))
        return larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list

    for i in range(len(points_list)): #finds the angles for all the tripods
        a = points_list[i][0]
        b = points_list[i][1]
        c = points_list[i][2]
        ang1 = getAngle(a, b, c) #all the three angles of the triangle comprised by the three points, three particles are measured
        ang2 = getAngle(b, a, c) #the selection of what point gets used is done...
        ang3 = getAngle(b, c, a) #wait for it
        ang = [ang1, ang2, ang3] #here
        tripod_ang = max(ang) #and here
        tripod_df['Tripod Angle'].iloc[i] = tripod_ang



    for i in range(len(points_list)): #finds the angles for the linear tripods and orders them regarding size and position
        a = points_list[i][0]
        b = points_list[i][1]
        c = points_list[i][2]
        ang1 = getAngle(a, b, c) #all the three angles of the triangle comprised by the three points, three particles are measured
        ang2 = getAngle(b, a, c) #the selection of what point gets used is done...
        ang3 = getAngle(b, c, a) #wait for it
        ang = [ang1, ang2, ang3] #here
        if max(ang) > 179.99:
            ang[0] = 179.99
            ang[1], ang[2] = 0.005, 0.005
        tripod_ang = max(ang) #and here
        tripod_df['Tripod Angle'].iloc[i] = tripod_ang
        if ang1 > ang2 and ang1 > ang3: #use the angle between the lobes to determine which one is the middle one and then also identifies which of the extrmities is the largest
            lobe_df['Middle Lobe'].iloc[lobe_tripod_list[i][1]] = True
            if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][0]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][2]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 0, 1, 2, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)
            else:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][2]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][0]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 2, 1, 0, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)
        if ang2 > ang1 and ang2 > ang3:
            lobe_df['Middle Lobe'].iloc[lobe_tripod_list[i][0]] = True
            if lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][1]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][2]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 1, 0, 2, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)
            else:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][2]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][1]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 2, 0, 1, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)
        if ang3 > ang1 and ang3 > ang2:
            lobe_df['Middle Lobe'].iloc[lobe_tripod_list[i][2]] = True
            if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]]:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][0]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][1]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 0, 2, 1, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)
            else:
                lobe_df['Large Extremity'].iloc[lobe_tripod_list[i][1]] = True
                lobe_df['Small Extremity'].iloc[lobe_tripod_list[i][0]] = True
                larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list = find_distances (points_list, i, 1, 2, 0, larger_to_middle_list, smaller_to_middle_list,  larger_to_smaller_list)

    ##########################################################
    ##########################################################
    ##########################################################
    tripod_df = tripod_df.reset_index()
    ##########################################################
    ##########################################################
    ##########################################################

    tripod_df['Large to Middle Distance'] = pd.Series(larger_to_middle_list)
    tripod_df['Small to Middle Distance'] = pd.Series(smaller_to_middle_list)
    tripod_df['Large to Small Distance'] = pd.Series(larger_to_smaller_list)
    largeextremity_df = lobe_df[lobe_df['Large Extremity']]
    middlelobe_df = lobe_df[lobe_df['Middle Lobe']]
    smallextremity_df = lobe_df[lobe_df['Small Extremity']]

    lobe_df['Larger'], lobe_df['Medium'], lobe_df['Smaller'] = False, False, False #classifies the lobes based on their sizes with no regard to their positioning within the tripod
    for i in range(len(lobe_tripod_list)):
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][2]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][2]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][1]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][2]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][1]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][2]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][2]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][0]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][0]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][1]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][1]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][0]] = True
        if lobe_df['Diameter'].iloc[lobe_tripod_list[i][2]] > lobe_df['Diameter'].iloc[lobe_tripod_list[i][0]] == lobe_df['Diameter'].iloc[lobe_tripod_list[i][1]]:
            lobe_df['Larger'].iloc[lobe_tripod_list[i][2]] = True
            lobe_df['Medium'].iloc[lobe_tripod_list[i][0]] = True
            lobe_df['Smaller'].iloc[lobe_tripod_list[i][1]] = True

    larger_df = lobe_df[lobe_df['Larger']]
    medium_df = lobe_df[lobe_df['Medium']]
    smaller_df = lobe_df[lobe_df['Smaller']]

    triangular_df = tripod_df[tripod_df['Classification'] == 'Triangular']
    linear_df = tripod_df[tripod_df['Classification'] == 'Linear']
    print ('Number of triangular tripods = ' + str(math.ceil(len(triangular_list))) + '; ' + str(round((len(triangular_list)*100/(len(linear_list)+len(triangular_list))),2)) + '%')
    print ('Number of linear tripods = ' + str(len(linear_list)) + '; ' + str(round(len(linear_list)*100/(len(linear_list)+len(triangular_list)),2)) + '%')

    return tripod_df, linear_df, triangular_df,  largeextremity_df, middlelobe_df, smallextremity_df, larger_df, medium_df, smaller_df

tripod_df, linear_df, triangular_df,  largeextremity_df, middlelobe_df, smallextremity_df, larger_df, medium_df, smaller_df = measure(tripod_df, lobe_df)

def treatdumbbells(dumbbell_df, lobe_df):
    lobe_positioning = []
    lobe_relationing = []
    copialista = []

    for i in range(len(lobe_df)): #function uses the distance between the dumbbell lobes and the whole dumbbells to determine to which dumbbells the lobes belong
        x = False
        for j in range(len(dumbbell_df)):
            if x == False:
                if lobe_df['Image'].iloc[i] == dumbbell_df['Image'].iloc[j] and dumbbell_df['X'].iloc[j] - dumbbell_df['Minor'].iloc[j]/2 < lobe_df['X'].iloc[i] < dumbbell_df['X'].iloc[j] + dumbbell_df['Minor'].iloc[j]/2 and dumbbell_df['Y'].iloc[j] - dumbbell_df['Minor'].iloc[j]/2 < lobe_df['Y'].iloc[i] < dumbbell_df['Y'].iloc[j] + dumbbell_df['Minor'].iloc[j]/2:
                    lobe_positioning.append([dumbbell_df['Image'].iloc[j], dumbbell_df['Particle'].iloc[j]])
                    lobe_relationing.append(dumbbell_df['Particle'].iloc[j])
                    x = True
                else:
                    x = False
        copialista.append(x)

    lobe_df = lobe_df[copialista] #drops all the spheres
    lobe_df.reset_index(drop=True, inplace=True)

    lobe_df['Dumbbell'] = lobe_relationing
    lobe_df['Large'] = False
    lobe_df['Small'] = False

    for i in range(len(lobe_positioning)): #identifies to which other half the lobe belongs
        for j in range(i+1, len(lobe_positioning)):
            if lobe_positioning[i] == lobe_positioning[j]:
                if lobe_df['Area'].iloc[i] > lobe_df['Area'].iloc[j]:
                    lobe_df['Large'].iloc[i] = True
                    lobe_df['Small'].iloc[j] = True
                if lobe_df['Area'].iloc[i] < lobe_df['Area'].iloc[j]:
                    lobe_df['Small'].iloc[i] = True
                    lobe_df['Large'].iloc[j] = True
                #print (str(i) + ',' + str(j))

    largedumbbelllobe_df = lobe_df[lobe_df['Large']]
    smalldumbbelllobe_df = lobe_df[lobe_df['Small']]

    return dumbbell_df, largedumbbelllobe_df, smalldumbbelllobe_df

dumbbell_df, largedumbbelllobe_df, smalldumbbelllobe_df = treatdumbbells(dumbbell_df, lobe_df)

def find_distances_dumbbells(dumbbell_df, largelobe_df, smalllobe_df):
    def dist (a, b): #calcula a distancia entre dois pontos por geometria euclidiana
        dist = math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
        return dist
    points_list = []
    for i in range(len(largelobe_df)):
        for j in range(len(smalllobe_df)): #acha qual o outro lado do dumbbell partindo do maior lado
            if largelobe_df['Image'].iloc[i] == smalllobe_df['Image'].iloc[j] and largelobe_df['Dumbbell'].iloc[i] == smalllobe_df['Dumbbell'].iloc[j]:
                points_list.append([largelobe_df['X'].iloc[i], largelobe_df['Y'].iloc[i], smalllobe_df['X'].iloc[j], smalllobe_df['Y'].iloc[j], largelobe_df['Image'].iloc[i], largelobe_df['Dumbbell'].iloc[i]])
    #aqui também são adicionados o numero da partícula e em qual imagem ela foi identificada

    distance_list = []
    for i in range(len(points_list)):
        distance_list.append(dist((points_list[i][0], points_list[i][1]),(points_list[i][2], points_list[i][3]))) #aqui as distancias sao medidas e a identificacao da particula entra na lista apos a ditancia

    dumbbell_df['Distance'] = ''
    for i in range(len(dumbbell_df)):
        for j in range(len(points_list)):
            if dumbbell_df['Image'].iloc[i] == points_list[j][4] and dumbbell_df['Particle'].iloc[i] == points_list[j][5]:
                dumbbell_df['Distance'][i] = distance_list[j]
        if dumbbell_df['Distance'].iloc[i] == '':
            dumbbell_df['Distance'].iloc[i] = np.NaN

    return dumbbell_df

dumbbell_df = find_distances_dumbbells(dumbbell_df, largedumbbelllobe_df, smalldumbbelllobe_df)

from collections import Counter
def samplemode(sample):
     c = Counter(sample)
     return [k for k, v in c.items() if v == c.most_common(1)[0][1]]

#creates a list with the most important parameters identified with the analysis
abstract_list = [['Total number of analyzed particles = ',round(math.ceil(len(dumbbell_df))+len(tripod_df), 1), ''],
                 ['Number of dumbbells = ', math.ceil(len(dumbbell_df)), ''],
                ['Fraction of dumbbells = ', round((len(dumbbell_df)*100/(len(tripod_df)+len(dumbbell_df))),2), '%'],
                 ['Number of tripods = ', len(tripod_df), ''],
                 ['Fraction of tripods = ', round(len(tripod_df)*100/(len(tripod_df)+len(dumbbell_df)),2), '%'],
                 ['Number of triangular tripods =', math.ceil(len(triangular_df)), '' ],
                 ['Fraction of triangular tripods = ', round((len(triangular_df)*100/(len(linear_df)+len(triangular_df))),2), '%'],
                 ['Number of linear tripods = ', len(linear_df), ''],
                 ['Fraction of linear tripods = ', round(len(linear_df)*100/(len(linear_df)+len(triangular_df)),2), '%']]

def addtoabstract (datalist, abstract_list, positioning):
    samplename = [['Mean Large Dumbbell Lobe Diameter = ', 'Large Dumbbell Lobe Standard Deviation = ', 'Large Dumbbell Lobe PDI = '],
                  ['Mean Small Dumbbell Lobe Diameter = ', 'Small Dumbbell Lobe Standard Deviation = ', 'Small Dumbbell Lobe PDI = '],
                  ['Mean Tripod Angle = ', 'Tripod Angle Standard Deviation = ', 'Tripod Angle Mode = '],
                  ['Mean Open Tripod Angle = ', 'Open Tripod Angle Standard Deviation = ', 'Open Tripod Angle Mode = '],
                  ['Mean Closed Tripod Angle = ', 'Closed Tripod Angle Standard Deviation = ', 'Closed Tripod Angle Mode = '],
                  ['Mean Large Tripod Extremity Diameter = ', 'Large Tripod Extremity Diameter Standard Deviation = ', 'Large Tripod Extremity Diameter PDI = '],
                  ['Mean Tripod Middle Lobe Diameter = ', 'Tripod Middle Lobe Diameter Standard Deviation = ', 'Mean Tripod Middle Lobe PDI = '],
                  ['Mean Small Tripod Extremity Diameter = ', 'Small Tripod Extremity Diameter Standard Deviation = ', 'Small Tripod Extremity Diameter PDI = '],
                  ['Mean Lobe Distance - Large to Middle = ', 'Lobe Distance - Large to Middle - Standard Deviation = ', 'Lobe Distance - Large to Middle - Mode = '],
                  ['Mean Lobe Distance - Small to Middle = ', 'Lobe Distance - Small to Middle - Standard Deviation = ', 'Lobe Distance - Small to Middle - Mode = '],
                  ['Mean Lobe Distance - Large to Small = ', 'Lobe Distance - Large to Small - Standard Deviation = ', 'Lobe Distance - Large to Small - Mode = '],
                  ['Mean Large Tripod Lobe Diameter = ', 'Large Tripod Lobe Diameter Standard Deviation = ', 'Large Tripod Lobe Diameter Mode = '],
                  ['Mean Medium Tripod Lobe Diameter = ', 'Medium Tripod Lobe Diameter Standard Deviation = ', 'Medium Tripod Lobe Diameter Mode = '],
                  ['Mean Small Tripod Lobe Diameter = ', 'Small Tripod Lobe Diameter Standard Deviation = ', 'Small Tripod Lobe Diameter Mode = '],
                  ['Mean Distance Between Dumbbell Lobes = ', 'Dumbbell Lobe Distance Standard Deviation = ', 'Distance Between Dumbbell Lobes Mode = ']]
    mean = statistics.mean(datalist)
    sd = statistics.stdev(datalist)
    mode = samplemode([round(num) for num in datalist])
    if 'Angle' in samplename[positioning][0]:
        abstract_list.append([samplename[positioning][0], mean, '°'])
        abstract_list.append([samplename[positioning][1], round(sd, 3), '°'])
    else:
        abstract_list.append([samplename[positioning][0], mean, 'nm'])
        abstract_list.append([samplename[positioning][1], round(sd, 3), 'nm'])

    PDIlist = [0,1,5,6,7]
    if positioning in PDIlist:
        datavolume = []
        for i in range(len(datalist)):
            datavolume.append((datalist.iloc[i]/2)**3)
        meanvol = statistics.mean(datavolume)
        meanvol = 2*(meanvol**(1/3))
        abstract_list.append([samplename[positioning][2], round(meanvol/mean, 7), ''])
    #for i in range(len(mode)):
    #    abstract_list.append([samplename[positioning][2], mode[i], 'nm'])
    return abstract_list

if len(dumbbell_df) > 0:
    abstract_list = addtoabstract(largedumbbelllobe_df['Diameter'], abstract_list, 0)
    abstract_list = addtoabstract(smalldumbbelllobe_df['Diameter'], abstract_list, 1)
    df = dumbbell_df.dropna()
    if len(df['Distance']) > 1:
        abstract_list = addtoabstract(df['Distance'], abstract_list, 14)

tripod_df = tripod_df.dropna()
if len(linear_df) > 0:
    abstract_list = addtoabstract(tripod_df['Tripod Angle'], abstract_list, 2)
    abstract_list = addtoabstract(linear_df['Tripod Angle'], abstract_list, 3)
if len(triangular_df) > 1:
    abstract_list = addtoabstract(triangular_df['Tripod Angle'], abstract_list, 4)
if len(tripod_df) > 0:
    abstract_list = addtoabstract(largeextremity_df['Diameter'], abstract_list, 5)
    abstract_list = addtoabstract(middlelobe_df['Diameter'], abstract_list, 6)
    abstract_list = addtoabstract(smallextremity_df['Diameter'], abstract_list, 7)
    abstract_list = addtoabstract(tripod_df['Large to Middle Distance'], abstract_list, 8)
    abstract_list = addtoabstract(tripod_df['Small to Middle Distance'], abstract_list, 9)
    abstract_list = addtoabstract(tripod_df['Large to Small Distance'], abstract_list, 9)
    abstract_list = addtoabstract(larger_df['Diameter'], abstract_list, 11)
    abstract_list = addtoabstract(medium_df['Diameter'], abstract_list, 12)
    abstract_list = addtoabstract(smaller_df['Diameter'], abstract_list, 13)

#abstract_list.append(['Large Lobe PDI = ', round((2*((3*mean(largelobe_df['Volume']))/(4*math.pi))**(1/3))/(mean(largelobe_df['Diameter'])), 7), 'nm'])

abstract_df = pd.DataFrame(abstract_list)


save_excel_tripods(abstract_df, tripod_df, linear_df, triangular_df,  largeextremity_df, middlelobe_df, smallextremity_df, larger_df, medium_df, smaller_df, dumbbell_df, largedumbbelllobe_df, smalldumbbelllobe_df)


abstract_df